In [ ]:
!pip install tkinter customtkinter
!pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import tkinter as tk
import customtkinter as ck

In [2]:
import mediapipe as mp
import numpy as np
import csv
import os
import pickle
import pandas as pd
import cv2
from matplotlib import pyplot as plt

#To set up the mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#Create array
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [3]:
# Load the .pkl file for the first set of data
with open('model1.pkl', 'rb') as f:
    model1 = pickle.load(f)

# Load the .pkl file for the second set of data
with open('model2.pkl', 'rb') as f:
    model2 = pickle.load(f)

def tracker(feed):
    counter = 0
    current_stage = ''
    cap = cv2.VideoCapture(feed)
    pause = False  # Flag to indicate whether the video is paused
    # up_accuracy = []
    # down_accuracy = []   
    # upbad_accuracy = []
    # downbad_accuracy = [] 
    # bend_accuracy = []   
    # cavein_accuracy = []  
    # caveout_accuracy = [] 
    # good_accuracy = []  

    # initiate holistic model
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            
            ret, frame = cap.read()

            if not ret:
                break
                
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make Detection
            results = pose.process(image)

            # Recolor image to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            landmark_color = (255, 255, 255)

            try:
                row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                X = pd.DataFrame([row], columns=landmarks[1:])
                body_language_class1 = model1.predict(X)[0]
                body_language_prob1 = model1.predict_proba(X)[0]
                print(body_language_class1, body_language_prob1)

                body_language_class2 = model2.predict(X)[0]
                body_language_prob2 = model2.predict_proba(X)[0]
                print(body_language_class2, body_language_prob2)
                
#                 if body_language_class1 == 'up':
#                     up_accuracy.append(body_language_prob1[np.argmax(body_language_prob1)])
#                 elif body_language_class1 == 'down':
#                     down_accuracy.append(body_language_prob1[np.argmax(body_language_prob1)])
#                 elif body_language_class1 == 'upbad':
#                     upbad_accuracy.append(body_language_prob1[np.argmax(body_language_prob1)])
#                 elif body_language_class1 == 'downbad':
#                     downbad_accuracy.append(body_language_prob1[np.argmax(body_language_prob1)])

#                 if body_language_class2 == 'bend':
#                     bend_accuracy.append(body_language_prob2[np.argmax(body_language_prob2)])
#                 elif body_language_class2 == 'cavein':
#                     cavein_accuracy.append(body_language_prob2[np.argmax(body_language_prob2)])
#                 elif body_language_class2 == 'caveout':
#                     caveout_accuracy.append(body_language_prob2[np.argmax(body_language_prob2)])
#                 elif body_language_class2 == 'good':
#                     good_accuracy.append(body_language_prob2[np.argmax(body_language_prob2)])
                    
                    
                    
                # to draw landmarks
                if body_language_class1 == 'downbad':
                    landmark_color = (0, 0, 255)  # Change color to red for 'downbad'
                elif body_language_class1 == 'upbad':
                    landmark_color = (0, 0, 255)  # Change color to green for 'upbad'
                    
                if body_language_class2 == 'bend':
                    landmark_color = (0, 0, 255)  # Change color to yellow for 'bend'
                elif body_language_class2 == 'cavein':
                    landmark_color = (0, 0, 255)  # Change color to cyan for 'cavein'
                elif body_language_class2 == 'caveout':
                     landmark_color = (0, 0, 255)  # Change color to magenta for 'caveout'

                        
                        
                # counter
                if body_language_class1 == 'up' and body_language_prob1[body_language_prob1.argmax()] >= .7:
                    current_stage = 'up'
                elif current_stage == 'up' and body_language_class1 == 'down' and body_language_prob1[body_language_prob1.argmax()] >= .7:
                    current_stage="down"
                    counter +=1
                    print(current_stage)
                    
                
                
                
                
#                 image = np.zeros((720, 1280, 3), dtype=np.uint8)

#                 # Calculate position to center the image on the canvas
#                 y_offset = (720 - image.shape[0]) // 2
#                 x_offset = (1280 - image.shape[1]) // 2

#                 # Overlay the image onto the canvas
#                 image[y_offset:y_offset + image.shape[0], x_offset:x_offset + image.shape[1]] = image


                # Render detections
                drawing_spec = mp_drawing.DrawingSpec(color=landmark_color, circle_radius=5)
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(245, 117, 16), thickness=2, circle_radius=2),
                                          connection_drawing_spec=drawing_spec
                                         )
                
                # if feed == 1:
                #     image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
                
                #Get status box
                cv2.rectangle (image, (0,0), (540, 60), (245, 117, 16), -1)


                

                #display probability
                cv2.putText(image, 'PROB'
                            , (10,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob1[np.argmax(body_language_prob1)],2))
                            , (15,40), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)
                
                #Display Up or Down
                cv2.putText(image, 'CLASS'
                            , (90,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, body_language_class1.split(' ')[0]
                            , (95,40), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)

                #display counter
                cv2.putText(image, 'COUNT'
                            , (220,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(counter)
                            , (225,40), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)

                #Display Class
                cv2.putText(image, 'FORM'
                            , (290,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, body_language_class2.split(' ')[0]
                            , (295,40), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)
                
                

                #display probability
                cv2.putText(image, 'FORM PROB'
                            , (420,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob2[np.argmax(body_language_prob2)],2))
                            , (425,40), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)
                

            except Exception as e:
                print("Error")
                pass
            
            # up_accuracy.append(0)
            # down_accuracy.append(0)
            # upbad_accuracy.append(0)
            # downbad_accuracy.append(0)
            # bend_accuracy.append(0)
            # cavein_accuracy.append(0)
            # caveout_accuracy.append(0)
            # good_accuracy.append(0)

            k = cv2.waitKey(1)
            if k == 27:  # press Esc key to exit
                break

            if not pause:
                #cv2.imshow('Mediapipe Feed', image)
                
                if feed == 1:

                    #image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
                    display_image = cv2.resize(image, (960, 720))
                    cv2.imshow('Mediapipe Feed', display_image)
                else:
                    display_image = cv2.resize(image, (1280, 720))
                    cv2.imshow('Mediapipe Feed', display_image)


        cap.release()
        cv2.destroyAllWindows()

        if cv2.waitKey(0) & 0xFF == ord('p'):
        # If 'p' is pressed after a video, pause before proceeding to the next video
            pause = True

In [ ]:
from tkinter import filedialog

ck.set_appearance_mode("Dark")
ck.set_default_color_theme("blue")

app = ck.CTk()
app.geometry("1280x720")
app.title("Motion Tracker")

    
def open_video():
    feed = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi;*.mkv")])
    if feed:
        tracker(feed)
    
def open_webcam():
    tracker(0)  # 0 represents the default camera (webcam)

title = ck.CTkLabel(app, text="Choose what to do", font=('Arial',40))
title.place(relx=0.5, rely=0.33, anchor='center')

button1 = ck.CTkButton(app, text="Video", command=open_video, font=('Arial', 40))
button1.place(relx=0.25, rely=0.67, anchor='center')

button2 = ck.CTkButton(app, text="Live", command=open_webcam, font=('Arial', 40))
button2.place(relx=0.75, rely=0.67, anchor='center')

app.mainloop()
